## Trip times

In [1]:
# display output from all cmds just like Python shell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import platform
print('python.version = ', platform.python_version())
import IPython
print('ipython.version =', IPython.version_info)

import numpy as np
print('numpy.version = ', np.__version__)
import pandas as pd
print('pandas.version = ', pd.__version__)

import sklearn
print('sklearn.version = ', sklearn.__version__)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib
print('matplotlib.version = ', matplotlib.__version__)
import matplotlib.pyplot as plt

import seaborn as sns
print('seaborn.version = ', sns.__version__)


python.version =  3.4.3
ipython.version = (6, 2, 1, '')
numpy.version =  1.13.3
pandas.version =  0.20.3
sklearn.version =  0.19.1
matplotlib.version =  2.1.0
seaborn.version =  0.8.1


## Import data

In [2]:
# read training set
train = pd.read_csv("../data/train.csv", index_col=0, header=0)
train.shape
train.info()

/usr/lib64/python3.4/dist-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(12905715, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12905715 entries, 0 to 12905714
Data columns (total 6 columns):
start_lng          float64
start_lat          float64
end_lng            float64
end_lat            float64
start_timestamp    int64
duration           int64
dtypes: float64(4), int64(2)
memory usage: 689.2 MB


In [39]:
# creat subset of training set for experimentation
t2 = train[:1000]
t2 = t2.copy(deep=True)
t2.head()
t2.shape


,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration
row_id,,,,,,
0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112
1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159
2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281
3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636
4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705


(1000, 6)

## Process timestamp

In [40]:
# split timestamp into components
utcdt = pd.to_datetime(t2['start_timestamp'], unit='s')
utcdt.head()

t2["year"] = utcdt.dt.year
t2["month"] = utcdt.dt.month
t2["dayofweek"] = utcdt.dt.weekday_name
t2["dayofmonth"] = utcdt.dt.day
t2["dayofyear"] = utcdt.dt.dayofyear
t2["hour"] = utcdt.dt.hour
t2["minute"] = utcdt.dt.minute

t2.head()


row_id
0   2015-01-11 04:33:39
1   2015-01-11 04:33:39
2   2015-01-16 03:05:41
3   2015-01-16 03:05:42
4   2015-01-25 08:13:06
Name: start_timestamp, dtype: datetime64[ns]

,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,year,month,dayofweek,dayofmonth,dayofyear,hour,minute
row_id,,,,,,,,,,,,,
0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,2015,1,Sunday,11,11,4,33
1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,2015,1,Sunday,11,11,4,33
2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,2015,1,Friday,16,16,3,5
3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,2015,1,Friday,16,16,3,5
4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,2015,1,Sunday,25,25,8,13


## Process location

In [43]:
t2["start"] = t2["start_lng"].map(str) + ',' + t2["start_lat"].map(str)
t2["end"] = t2["end_lng"].map(str) + ',' + t2["end_lat"].map(str)
t2.head()

t2["start"].value_counts()
t2["end"].value_counts()


,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,year,month,dayofweek,dayofmonth,dayofyear,hour,minute,start,stop,end
row_id,,,,,,,,,,,,,,,,
0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,2015,1,Sunday,11,11,4,33,"-74.009087,40.713817999999996","-74.004326,40.719986","-74.004326,40.719986"
1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,2015,1,Sunday,11,11,4,33,"-73.971176,40.762428","-74.004181,40.742653000000004","-74.004181,40.742653000000004"
2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,2015,1,Friday,16,16,3,5,"-73.994957,40.745079","-73.999939,40.73465","-73.999939,40.73465"
3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,2015,1,Friday,16,16,3,5,"-73.991127,40.75008","-73.988609,40.73489","-73.988609,40.73489"
4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,2015,1,Sunday,25,25,8,13,"-73.945511,40.773724","-73.987434,40.755707","-73.987434,40.755707"


-73.965637,40.762035                     1
-73.975594,40.776852000000005            1
-73.964439,40.77002                      1
-74.005882,40.706291                     1
-73.988419,40.721546000000004            1
-73.959091,40.783287                     1
-73.97496,40.790428000000006             1
-73.978195,40.783478                     1
-73.97177099999999,40.762980999999996    1
-73.99675,40.723106                      1
-73.99176800000001,40.749390000000005    1
-73.971863,40.757042                     1
-73.927689,40.692710999999996            1
-73.987747,40.770103000000006            1
-73.973663,40.7794                       1
-73.995628,40.759277000000004            1
-73.969864,40.750510999999996            1
-73.984932,40.743384999999996            1
-73.980659,40.76487                      1
-73.994164,40.750984                     1
-74.008255,40.714420000000004            1
-73.980522,40.745461                     1
-73.97403,40.763126                      1
-73.992073,

-74.007568,40.740959000000004            2
-73.98921999999999,40.738571             1
-73.95590200000001,40.766117             1
-73.993057,40.692932                     1
-73.971313,40.755013                     1
-73.97036,40.762375                      1
-73.864929,40.770432                     1
-73.975594,40.717262                     1
-73.993309,40.749779                     1
-73.98281899999999,40.7019               1
-73.984337,40.748131                     1
-73.96171600000001,40.761993             1
-73.964737,40.760246                     1
-73.967674,40.792614                     1
-73.961647,40.777122                     1
-73.980469,40.754028000000005            1
-73.985222,40.746189                     1
-73.95766400000001,40.770344             1
-73.97135899999999,40.760433             1
-73.991364,40.749893                     1
-73.941986,40.842842                     1
-74.013603,40.715015                     1
-73.95474200000001,40.780097999999995    1
-73.872322,

## Process duration

## Process missing values